In [466]:
import numpy as np
import pandas as pd
from sklearn.metrics import r2_score
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor, GradientBoostingRegressor
from sklearn.model_selection import train_test_split

In [467]:
train = pd.read_csv('train.csv', delimiter=',')
y = pd.DataFrame(train)

In [468]:
#Подготовка информации TRAIN

In [469]:
# Исправляем в трейне не верно указанные годы
train.loc[1497, 'HouseYear'] = (2005+2011)/2
train.loc[4189, 'HouseYear'] = 1968
train.describe()

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Social_1,Social_2,Social_3,Healthcare_1,Helthcare_2,Shops_1,Price
count,10000.00000,10000.000000,10000.000000,10000.000000,7887.000000,10000.000000,10000.000000,10000.000000,10000.00000,10000.000000,10000.000000,10000.000000,10000.000000,5202.000000,10000.000000,10000.000000,10000.000000
mean,8383.40770,50.400800,1.890500,56.315775,37.199645,6.273300,8.526700,12.609400,1984.86600,0.118858,24.687000,5352.157400,8.039200,1142.904460,1.319500,4.231300,214138.857399
std,4859.01902,43.587592,0.839512,21.058732,86.241209,28.560917,5.241148,6.775974,18.41187,0.119025,17.532614,4006.799803,23.831875,1021.517264,1.493601,4.806341,92872.293865
min,0.00000,0.000000,0.000000,1.136859,0.370619,0.000000,1.000000,0.000000,1910.00000,0.000000,0.000000,168.000000,0.000000,0.000000,0.000000,0.000000,59174.778028
25%,4169.50000,20.000000,1.000000,41.774881,22.769832,1.000000,4.000000,9.000000,1974.00000,0.017647,6.000000,1564.000000,0.000000,350.000000,0.000000,1.000000,153872.633942
50%,8394.50000,36.000000,2.000000,52.513310,32.781260,6.000000,7.000000,13.000000,1977.00000,0.075424,25.000000,5285.000000,2.000000,900.000000,1.000000,3.000000,192269.644879
75%,12592.50000,75.000000,2.000000,65.900625,45.128803,9.000000,12.000000,17.000000,2001.00000,0.195781,36.000000,7227.000000,5.000000,1548.000000,2.000000,6.000000,249135.462171
max,16798.00000,209.000000,19.000000,641.065193,7480.592129,2014.000000,42.000000,117.000000,2020.00000,0.521867,74.000000,19083.000000,141.000000,4849.000000,6.000000,23.000000,633233.466570


In [470]:
#Выбираем строки с данными, без нан
c=train[train['LifeSquare'].notna()]
c=c[c['Healthcare_1'].notna()]
c.head()

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Ecology_2,Ecology_3,Social_1,Social_2,Social_3,Healthcare_1,Helthcare_2,Shops_1,Shops_2,Price
1,15053,41,3.0,65.683640,40.049543,8.0,7,9.0,1978.0,0.000070,B,B,46,10309,1,240.0,1,16,B,300009.450063
2,4765,53,2.0,44.947953,29.197612,0.0,8,12.0,1968.0,0.049637,B,B,34,7759,0,229.0,1,3,B,220925.908524
3,5809,58,2.0,53.352981,52.731512,9.0,8,17.0,1977.0,0.437885,B,B,23,5735,3,1084.0,0,5,B,175616.227217
4,10783,99,1.0,39.649192,23.776169,7.0,11,12.0,1976.0,0.012339,B,B,35,5776,1,2078.0,2,4,B,150226.531644
5,12915,59,3.0,80.384479,46.683720,12.0,5,17.0,2011.0,0.309479,B,B,35,7715,4,990.0,0,6,B,215898.447742


In [471]:
#Смотрим выбросы в lifesquare
d=train[train['LifeSquare'] >= 400].sort_values(by = 'LifeSquare', axis=0)
d

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Ecology_2,Ecology_3,Social_1,Social_2,Social_3,Healthcare_1,Helthcare_2,Shops_1,Shops_2,Price
4690,2307,102,1.0,409.425181,410.639749,10.0,4,4.0,2016.0,0.238617,B,B,26,3889,6,705.0,3,6,B,90470.430830
8437,15886,85,3.0,78.059331,461.463614,10.0,12,16.0,1998.0,0.037178,B,B,52,11217,1,2300.0,1,7,B,394253.299978
6977,11602,30,2.0,641.065193,638.163193,10.0,20,19.0,2019.0,0.000078,B,B,22,6398,141,1046.0,3,23,B,133529.681562
4328,16550,27,3.0,81.694417,7480.592129,1.0,9,17.0,2016.0,0.017647,B,B,2,469,0,NaN,0,0,B,217357.492366


In [472]:
#Удаляем выбросы d LifeSquare из train так как все из отобраных - ошибочные
train=train.drop(d.index, axis = 0)
train.describe()

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Social_1,Social_2,Social_3,Healthcare_1,Helthcare_2,Shops_1,Price
count,9996.000000,9996.000000,9996.000000,9996.000000,7883.000000,9996.000000,9996.000000,9996.000000,9996.000000,9996.000000,9996.000000,9996.00000,9996.000000,5199.000000,9996.000000,9996.000000,9996.000000
mean,8382.126050,50.396559,1.890356,56.217238,36.077983,6.272709,8.525610,12.608844,1984.855042,0.118876,24.686675,5352.10094,8.027611,1142.784766,1.319328,4.229392,214140.952689
std,4858.238159,43.590777,0.839485,19.920870,18.014430,28.566510,5.240627,6.776254,18.406648,0.119030,17.532500,4006.83403,23.799261,1021.667012,1.493649,4.803336,92861.659213
min,0.000000,0.000000,0.000000,1.136859,0.370619,0.000000,1.000000,0.000000,1910.000000,0.000000,0.000000,168.00000,0.000000,0.000000,0.000000,0.000000,59174.778028
25%,4169.500000,20.000000,1.000000,41.773051,22.767925,1.000000,4.000000,9.000000,1974.000000,0.017647,6.000000,1564.00000,0.000000,350.000000,0.000000,1.000000,153891.653482
50%,8393.500000,36.000000,2.000000,52.502558,32.767366,6.000000,7.000000,13.000000,1977.000000,0.075424,25.000000,5285.00000,2.000000,900.000000,1.000000,3.000000,192269.644879
75%,12591.250000,75.000000,2.000000,65.889736,45.107755,9.000000,12.000000,17.000000,2001.000000,0.195781,36.000000,7227.00000,5.000000,1548.000000,2.000000,6.000000,249135.462171
max,16798.000000,209.000000,19.000000,604.705972,263.542020,2014.000000,42.000000,117.000000,2020.000000,0.521867,74.000000,19083.00000,141.000000,4849.000000,6.000000,23.000000,633233.466570


In [473]:
#Смотрим этажи и высоты домов
dflo=train[train['HouseFloor'] <= train['Floor']]

# 2727 несоответсующих позиций, но часть из них жилье на последних этажах
#их надо убрать из этой выборки

In [474]:
#Выборка квартир на последних этажах
dflo1=train[train['HouseFloor'] == train['Floor']]


In [475]:
#Убираем из выбрки несоответствия этажности квартиры на последних этажах
dflo=dflo.drop(dflo1.index, axis = 0)

In [476]:
#Так как выборка слишком велика, чтобы ее дропнуть, 
#то меняем местами этажность и этаж квартиры
dflo['temp']=dflo['Floor']
dflo['Floor']=dflo['HouseFloor']


In [477]:
# Удаляем из трейна строки с неправильными этажами и присоединяем с правильными
train=train.drop(dflo.index, axis = 0)
train=pd.concat([train, dflo], axis = 0)
train=train.drop('temp', axis = 1)
train.describe()


,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Social_1,Social_2,Social_3,Healthcare_1,Helthcare_2,Shops_1,Price
count,9996.000000,9996.000000,9996.000000,9996.000000,7883.000000,9996.000000,9996.000000,9996.000000,9996.000000,9996.000000,9996.000000,9996.00000,9996.000000,5199.000000,9996.000000,9996.000000,9996.000000
mean,8382.126050,50.396559,1.890356,56.217238,36.077983,6.272709,7.699080,12.608844,1984.855042,0.118876,24.686675,5352.10094,8.027611,1142.784766,1.319328,4.229392,214140.952689
std,4858.238159,43.590777,0.839485,19.920870,18.014430,28.566510,5.303598,6.776254,18.406648,0.119030,17.532500,4006.83403,23.799261,1021.667012,1.493649,4.803336,92861.659213
min,0.000000,0.000000,0.000000,1.136859,0.370619,0.000000,0.000000,0.000000,1910.000000,0.000000,0.000000,168.00000,0.000000,0.000000,0.000000,0.000000,59174.778028
25%,4169.500000,20.000000,1.000000,41.773051,22.767925,1.000000,4.000000,9.000000,1974.000000,0.017647,6.000000,1564.00000,0.000000,350.000000,0.000000,1.000000,153891.653482
50%,8393.500000,36.000000,2.000000,52.502558,32.767366,6.000000,7.000000,13.000000,1977.000000,0.075424,25.000000,5285.00000,2.000000,900.000000,1.000000,3.000000,192269.644879
75%,12591.250000,75.000000,2.000000,65.889736,45.107755,9.000000,11.000000,17.000000,2001.000000,0.195781,36.000000,7227.00000,5.000000,1548.000000,2.000000,6.000000,249135.462171
max,16798.000000,209.000000,19.000000,604.705972,263.542020,2014.000000,42.000000,117.000000,2020.000000,0.521867,74.000000,19083.00000,141.000000,4849.000000,6.000000,23.000000,633233.466570


In [478]:
#Смотрим этажи и высоты домов 2 проверка
dflo2=train[train['HouseFloor'] <= train['Floor']]


In [479]:
#Выбираем строки с данными, без нан, чтобы посчитать коэффициенты и средние для вставки вместо нан
nadrp=train[train['LifeSquare'].notna()]
nadrp=nadrp[nadrp['Healthcare_1'].notna()]


In [480]:
(nadrp['LifeSquare'] / nadrp['Square']).mean()

0.6349049298116568

In [481]:
nadrp['Healthcare_1'].median()

1015.0

In [482]:
#Заменяем нан 
c2=train[train['LifeSquare'].isna()]
c2.loc[c2['LifeSquare'].isna(), 'LifeSquare']=c2['Square']*0.634
c2=c2[c2['Healthcare_1'].isna()]
c2.loc[c2['Healthcare_1'].isna(), 'Healthcare_1']=1015
c2.isna().sum(axis =0)

C:\Anaconda3\lib\site-packages\pandas\core\indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


Id               0
DistrictId       0
Rooms            0
Square           0
LifeSquare       0
KitchenSquare    0
Floor            0
HouseFloor       0
HouseYear        0
Ecology_1        0
Ecology_2        0
Ecology_3        0
Social_1         0
Social_2         0
Social_3         0
Healthcare_1     0
Helthcare_2      0
Shops_1          0
Shops_2          0
Price            0
dtype: int64

In [483]:
# Меняем в трейне наны на данные с коффициентом по площади и на медианные по здооровью
train.loc[train['Healthcare_1'].isna(), 'Healthcare_1']=1015
train.loc[train['LifeSquare'].isna(), 'LifeSquare'] = train['Square']*0.634
#train.isna().sum(axis =0)

In [484]:
#Анализируем площадь кухни
kitchsqu=train[train['KitchenSquare'] >= 4].sort_values(by = 'KitchenSquare', axis=0)
kitchsqu=kitchsqu[kitchsqu['KitchenSquare'] <= 16].sort_values(by = 'KitchenSquare', axis=0)
kitchsqu.head()
#проверяем корреляцию кухни с другими признаками, площадь, этажность дома, год постройки поэтому из наиболее представительной выборки
#с метражем кухни от 4 до 16 посчитаем коэффициент и применим к остальной выборке
#

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Ecology_2,Ecology_3,Social_1,Social_2,Social_3,Healthcare_1,Helthcare_2,Shops_1,Shops_2,Price
6846,5313,175,4.0,78.204702,63.867593,4.0,5.0,5.0,1933.0,0.037316,B,B,32,6748,93,562.0,4,10,B,465934.434276
7954,1943,23,1.0,34.917015,23.916309,4.0,3.0,9.0,1977.0,0.075779,B,B,6,1437,3,1015.0,0,2,B,123381.294367
6342,8906,84,1.0,35.992431,23.677642,4.0,5.0,5.0,1960.0,0.006076,B,B,30,5285,0,645.0,6,6,B,158302.141137
4400,10495,108,1.0,30.826070,18.854840,4.0,3.0,8.0,1959.0,0.000000,B,B,25,5027,4,46.0,1,1,B,181626.175103
1343,11145,8,2.0,43.392053,31.550798,4.0,3.0,5.0,1967.0,0.458174,B,A,26,4439,1,1130.0,1,5,B,160868.809677


In [485]:
# Расчет среднего коэффициента соотношения площади всего и кухни
(kitchsqu['KitchenSquare'] / kitchsqu['Square']).mean()
#кухня в среднем = 15 процентов от общей площади - будем использовать этот коэффициент

0.15390242034403986

In [486]:
# Меняем в трейне кухни с площадью меньше 4 и больше 16 на данные с коффициентом 0,1543 по общей площади 
#kitchsqu=train[train['KitchenSquare'] >= 4].sort_values(by = 'KitchenSquare', axis=0)
#kitchsqu=kitchsqu[kitchsqu['KitchenSquare'] <= 16].sort_values(by = 'KitchenSquare', axis=0)
#kitchsqu
train.loc[train['KitchenSquare'] <= 4, 'KitchenSquare']=train['Square']*0.1543
train.loc[train['KitchenSquare'] >= 16, 'KitchenSquare']=train['Square']*0.1543
train.describe()

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Social_1,Social_2,Social_3,Healthcare_1,Helthcare_2,Shops_1,Price
count,9996.000000,9996.000000,9996.000000,9996.000000,9996.000000,9996.000000,9996.000000,9996.000000,9996.000000,9996.000000,9996.000000,9996.00000,9996.000000,9996.000000,9996.000000,9996.000000,9996.000000
mean,8382.126050,50.396559,1.890356,56.217238,36.539984,8.387278,7.699080,12.608844,1984.855042,0.118876,24.686675,5352.10094,8.027611,1081.461885,1.319328,4.229392,214140.952689
std,4858.238159,43.590777,0.839485,19.920870,17.350628,2.848883,5.303598,6.776254,18.406648,0.119030,17.532500,4006.83403,23.799261,739.538040,1.493649,4.803336,92861.659213
min,0.000000,0.000000,0.000000,1.136859,0.370619,0.175417,0.000000,0.000000,1910.000000,0.000000,0.000000,168.00000,0.000000,0.000000,0.000000,0.000000,59174.778028
25%,4169.500000,20.000000,1.000000,41.773051,24.449867,6.007884,4.000000,9.000000,1974.000000,0.017647,6.000000,1564.00000,0.000000,830.000000,0.000000,1.000000,153891.653482
50%,8393.500000,36.000000,2.000000,52.502558,33.360384,8.000000,7.000000,13.000000,1977.000000,0.075424,25.000000,5285.00000,2.000000,1015.000000,1.000000,3.000000,192269.644879
75%,12591.250000,75.000000,2.000000,65.889736,44.994860,10.000000,11.000000,17.000000,2001.000000,0.195781,36.000000,7227.00000,5.000000,1015.000000,2.000000,6.000000,249135.462171
max,16798.000000,209.000000,19.000000,604.705972,383.383586,93.306131,42.000000,117.000000,2020.000000,0.521867,74.000000,19083.00000,141.000000,4849.000000,6.000000,23.000000,633233.466570


In [487]:
#анализируем нулевую площадь
square=train[train['Square'] <= 6].sort_values(by = 'Square', axis=0)
square


,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Ecology_2,Ecology_3,Social_1,Social_2,Social_3,Healthcare_1,Helthcare_2,Shops_1,Shops_2,Price
6392,14786,1,1.0,1.136859,4.525736,0.175417,1.0,1.0,1977.0,0.007122,B,B,1,264,0,1015.0,0,1,B,181434.825589
8283,15744,34,1.0,1.988943,2.642219,0.306894,4.0,4.0,1977.0,0.069753,B,B,53,13670,4,1015.0,1,11,B,458378.777006
4853,3224,27,0.0,2.377248,0.873147,0.366809,0.0,0.0,1977.0,0.017647,B,B,2,469,0,1015.0,0,0,B,126596.941798
1608,10202,6,1.0,2.596351,4.604943,0.400617,3.0,25.0,2014.0,0.243205,B,B,5,1564,0,540.0,0,0,B,137597.601458
9294,6782,45,1.0,2.954309,5.257278,0.455850,1.0,1.0,1977.0,0.195781,B,B,23,5212,6,1015.0,3,2,B,438005.182323
3280,10527,27,1.0,4.380726,40.805837,0.675946,10.0,17.0,2013.0,0.211401,B,B,9,1892,0,1015.0,0,1,B,97560.720383
4900,4504,27,3.0,4.390331,5.610772,0.677428,8.0,19.0,2016.0,0.211401,B,B,9,1892,0,1015.0,0,1,B,161379.067034
9613,7657,38,1.0,4.502784,4.618547,0.694780,1.0,1.0,1977.0,0.060753,B,B,15,2787,2,520.0,0,7,B,483283.488083
1316,11526,27,1.0,4.633498,1.969969,0.714949,1.0,1.0,1977.0,0.011654,B,B,4,915,0,1015.0,0,0,B,107604.269441
8030,13265,1,3.0,4.823679,79.767964,0.744294,6.0,17.0,1977.0,0.007122,B,B,1,264,0,1015.0,0,1,B,237716.681261


In [488]:
#Удаляем из выбрки ошибочные данные квартиры с метражем меньше 10 кв. м.
train=train.drop(square.index, axis = 0)

In [489]:
#Смотрим нулевую площадь
square2=train[train['Square'] >= train['LifeSquare']].sort_values(by = 'Square', axis=0)
# Расчет среднего коэффициента соотношения площади всего и жилой площади
(square2['LifeSquare'] / square2['Square']).mean()
#заменяем в выборке не верно указанную жилую площадь
train.loc[train['Square'] <= train['LifeSquare'], 'LifeSquare']=train['Square']*(square2['LifeSquare'] / square2['Square']).mean()
square3=train[train['Square'] >= train['LifeSquare']].sort_values(by = 'Square', axis=0) 
square3                                  

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Ecology_2,Ecology_3,Social_1,Social_2,Social_3,Healthcare_1,Helthcare_2,Shops_1,Shops_2,Price
4739,12676,81,3.0,13.784865,8.618068,7.000000,4.0,5.0,1960.0,0.319809,B,B,25,4756,16,2857.0,5,8,B,78388.806186
7767,6927,21,1.0,16.117154,8.822988,2.486877,2.0,5.0,1965.0,0.194489,B,B,47,8004,3,125.0,3,5,B,79913.267856
5617,15877,57,3.0,16.547242,10.345060,8.000000,5.0,5.0,1962.0,0.133215,B,B,49,11395,3,1406.0,3,4,A,77835.185808
3166,15127,150,1.0,17.046188,10.656992,8.000000,7.0,12.0,1982.0,0.300323,B,B,52,10311,6,1015.0,1,9,B,74222.110465
6945,12927,62,2.0,17.414181,11.040591,2.687008,4.0,20.0,1977.0,0.072158,B,B,2,629,1,1015.0,0,0,A,180750.471749
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8511,4249,23,5.0,198.930182,123.139863,10.000000,1.0,2.0,2011.0,0.014073,B,B,2,475,0,1015.0,0,0,B,345665.209100
9910,16568,27,4.0,200.334539,125.245813,30.911619,1.0,2.0,2013.0,0.041116,B,B,53,14892,4,1015.0,1,4,B,528560.506016
1981,7917,27,0.0,212.932361,211.231125,32.855463,2.0,3.0,2008.0,0.211401,B,B,9,1892,0,1015.0,0,1,B,302211.260887
1982,5548,86,5.0,275.645284,233.949309,42.532067,12.0,37.0,2011.0,0.161976,B,A,31,7010,5,4508.0,3,7,B,455264.882666


In [490]:
# Убираем буквы в признаках
train.loc[train['Ecology_2'] == 'A', 'Ecology_2'] = 0
train.loc[train['Ecology_2'] == 'B', 'Ecology_2'] = 1
train.loc[train['Ecology_3'] == 'A', 'Ecology_3'] = 0
train.loc[train['Ecology_3'] == 'B', 'Ecology_3'] = 1
train.loc[train['Shops_2'] == 'A', 'Shops_2'] = 0
train.loc[train['Shops_2'] == 'B', 'Shops_2'] = 1

In [491]:
#Смена типа данных в выборке
train['Ecology_2']=train['Ecology_2'].astype(int)
train['Ecology_3']=train['Ecology_3'].astype(int)
train['Shops_2']=train['Shops_2'].astype(int)

In [492]:
# Конец подготовки данных

In [493]:
#Готовим и обучаем модель на TRAIN
prce=train["Price"]
y=pd.DataFrame(prce,columns=["Price"])
y.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9984 entries, 0 to 9995
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Price   9984 non-null   float64
dtypes: float64(1)
memory usage: 476.0 KB


In [494]:
x=pd.DataFrame(train, columns=[ 'DistrictId', 'Rooms', 'Square', 'LifeSquare', 'KitchenSquare',
       'Floor', 'HouseFloor', 'HouseYear', 'Ecology_1', 'Ecology_2',
       'Ecology_3', 'Social_1', 'Social_2', 'Social_3', 'Healthcare_1',
       'Helthcare_2', 'Shops_1', 'Shops_2'])

In [495]:
#Разбиваем трейн на трейн и валид
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(x, y, test_size = 0.02, random_state=100)

In [496]:
X_train.shape

(9784, 18)

In [497]:
#Обучаем модель и считаем R2
clf = RandomForestRegressor(max_depth = 15,n_estimators=200, random_state=100)
clf.fit(X_train, y_train)
y_pred1 = clf.predict(X_valid)
    #y_pred_train = clf.predict(X_train)
r2_score(y_valid, y_pred1)


C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until


0.7855913152219305

In [498]:
y_pred1

array([ 99590.87619227, 173327.80757448, 189911.7699684 , 113185.02878048,
       210225.66936462, 316701.74205881, 181960.34271902, 141371.53305377,
       239161.53763161, 208548.85548059, 268145.4977452 , 237536.05323922,
       273814.52715024, 115710.08873114, 246521.01255904, 209300.42015707,
       241433.74619038, 197312.02964074, 309103.07118379, 229837.88413772,
       291832.29844367, 209985.70163055, 233829.29101605, 184740.10576157,
       381319.99020182,  73886.8578354 , 139406.50516106, 179603.60077178,
       245629.53261589, 259211.10123178,  71742.81235863, 120055.67365738,
       492608.73706307, 201768.68202743, 150700.76173958, 222094.49108702,
       171636.40244345, 231891.88230165, 105787.54010674, 233544.00090029,
       182726.5045945 , 204840.55036849, 218007.52201794, 286420.44258701,
       199533.21085817, 143695.83325718, 236614.04152176, 138494.25019324,
       433571.58189685, 425563.51486564, 348351.85311101, 125213.70910026,
       273460.83922745, 2

In [499]:
#Обучаем на полной выборке трейна
prce=train["Price"]
y=pd.DataFrame(prce,columns=["Price"])
x=pd.DataFrame(train, columns=[ 'DistrictId', 'Rooms', 'Square', 'LifeSquare', 'KitchenSquare',
       'Floor', 'HouseFloor', 'HouseYear', 'Ecology_1', 'Ecology_2',
       'Ecology_3', 'Social_1', 'Social_2', 'Social_3', 'Healthcare_1',
       'Helthcare_2', 'Shops_1', 'Shops_2'])
clf2 = RandomForestRegressor(max_depth = 15,n_estimators=200, random_state=100)
clf2.fit(x, y)
y_pred2 = clf2.predict(x)
r2_score(y, y_pred2)


C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  if __name__ == '__main__':


0.9340266594687195

In [500]:
#Подготовка информации для прогноза на TEST
test = pd.read_csv('test.csv', delimiter=',')
y = pd.DataFrame(test)

In [501]:
test_obg_feat = test.select_dtypes(include='object')
test_obg_feat.shape

(5000, 3)

In [502]:
test.isna().sum(axis =0)

Id                  0
DistrictId          0
Rooms               0
Square              0
LifeSquare       1041
KitchenSquare       0
Floor               0
HouseFloor          0
HouseYear           0
Ecology_1           0
Ecology_2           0
Ecology_3           0
Social_1            0
Social_2            0
Social_3            0
Healthcare_1     2377
Helthcare_2         0
Shops_1             0
Shops_2             0
dtype: int64

In [503]:
#Смотрим выбросы в lifesquare
d=test[test['LifeSquare'] <=9 ].sort_values(by = 'LifeSquare', axis=0)
d.head()

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Ecology_2,Ecology_3,Social_1,Social_2,Social_3,Healthcare_1,Helthcare_2,Shops_1,Shops_2
78,6553,34,5.0,122.375273,0.333490,0.0,9,0.0,1977,0.069753,B,B,53,13670,4,NaN,1,11,B
2992,6406,27,2.0,66.469510,0.567267,0.0,4,0.0,2018,0.017647,B,B,2,469,0,NaN,0,0,B
2341,3028,9,1.0,43.348348,0.809265,0.0,16,17.0,2015,0.161532,B,B,25,5648,1,30.0,2,4,B
2398,13014,53,3.0,81.141636,0.863989,0.0,10,1.0,2017,0.049637,B,B,34,7759,0,229.0,1,3,B
326,14959,1,2.0,64.112441,1.181139,1.0,13,17.0,1977,0.007122,B,B,1,264,0,NaN,0,1,B


In [504]:
#Смотрим этажи и высоты домов
dflo=test[test['HouseFloor'] <= test['Floor']]
dflo.shape
# 1391 несоответсующих позиций, но часть из них жилье на последних этажах
#их надо убрать из этой выборки

(1391, 19)

In [505]:
#Выборка квартир на последних этажах
dflo1=test[test['HouseFloor'] == test['Floor']]
#Убираем из выбрки несоответствия этажности квартиры на последних этажах
dflo=dflo.drop(dflo1.index, axis = 0)

In [506]:
#Так как выборка слишком велика, чтобы ее дропнуть, 
#то меняем местами этажность и этаж квартиры
dflo['temp']=dflo['Floor']
dflo['Floor']=dflo['HouseFloor']
# Удаляем из теста строки с неправильными этажами и присоединяем с правильными
test=test.drop(dflo.index, axis = 0)
test=pd.concat([test, dflo], axis = 0)
test=test.drop('temp', axis = 1)
test.describe()

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Social_1,Social_2,Social_3,Healthcare_1,Helthcare_2,Shops_1
count,5000.000000,5000.000000,5000.000000,5000.000000,3959.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,2623.000000,5000.00000,5000.000000
mean,8412.595400,51.279200,1.910000,56.449500,36.158810,5.976800,7.838000,12.601000,1984.392600,0.119874,24.933800,5406.900000,8.262600,1146.657263,1.31940,4.242800
std,4832.674037,44.179466,0.838594,19.092787,17.825287,9.950018,5.454989,6.789213,18.573149,0.120070,17.532202,4026.614773,23.863762,1044.744231,1.47994,4.777365
min,1.000000,0.000000,0.000000,1.378543,0.333490,0.000000,0.000000,0.000000,1908.000000,0.000000,0.000000,168.000000,0.000000,0.000000,0.00000,0.000000
25%,4221.750000,21.000000,1.000000,41.906231,23.092026,1.000000,4.000000,9.000000,1973.000000,0.019509,6.000000,1564.000000,0.000000,325.000000,0.00000,1.000000
50%,8320.500000,37.000000,2.000000,52.921340,32.925087,6.000000,7.000000,12.000000,1977.000000,0.072158,25.000000,5285.000000,2.000000,900.000000,1.00000,3.000000
75%,12598.250000,77.000000,2.000000,66.285129,45.174091,9.000000,11.000000,17.000000,2000.000000,0.195781,36.000000,7287.000000,5.000000,1548.000000,2.00000,6.000000
max,16795.000000,212.000000,17.000000,223.453689,303.071094,620.000000,46.000000,99.000000,2020.000000,0.521867,74.000000,19083.000000,141.000000,4849.000000,6.00000,23.000000


In [507]:
#Смотрим этажи и высоты домов 2 проверка
dflo2=test[test['HouseFloor'] <= test['Floor']]
dflo2.head()

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Ecology_2,Ecology_3,Social_1,Social_2,Social_3,Healthcare_1,Helthcare_2,Shops_1,Shops_2
3,15664,47,2.0,73.046609,51.940842,9.0,22.0,22.0,2007,0.101872,B,B,23,4583,3,NaN,3,3,B
4,14275,27,1.0,47.527111,43.387569,1.0,17.0,17.0,2017,0.072158,B,B,2,629,1,NaN,0,0,A
5,7633,53,1.0,40.675627,NaN,1.0,21.0,21.0,1977,0.049637,B,B,34,7759,0,229.0,1,3,B
8,4220,96,3.0,72.277106,45.968758,9.0,17.0,17.0,1997,0.041116,B,B,53,14892,4,NaN,1,4,B
16,13543,167,3.0,80.220720,51.687097,7.0,5.0,5.0,1959,0.050272,B,B,21,4231,14,1020.0,0,3,B


In [508]:
#Выбираем строки с данными, без нан, чтобы посчитать коэффициенты и средние для вставки вместо нан
nadrp=test[test['LifeSquare'].notna()]
nadrp=nadrp[nadrp['Healthcare_1'].notna()]

In [509]:
ss=(nadrp['LifeSquare'] / nadrp['Square']).mean()
ss

0.6475910839455888

In [510]:
aa=nadrp['Healthcare_1'].median()
aa

990.0

In [511]:
#Выбираем строки с данными, с нан
c2=test[test['LifeSquare'].isna()]
c2.loc[c2['LifeSquare'].isna(), 'LifeSquare']=c2['Square']*ss
c2=c2[c2['Healthcare_1'].isna()]
c2.loc[c2['Healthcare_1'].isna(), 'Healthcare_1']=aa
c2.isna().sum(axis =0)

C:\Anaconda3\lib\site-packages\pandas\core\indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


Id               0
DistrictId       0
Rooms            0
Square           0
LifeSquare       0
KitchenSquare    0
Floor            0
HouseFloor       0
HouseYear        0
Ecology_1        0
Ecology_2        0
Ecology_3        0
Social_1         0
Social_2         0
Social_3         0
Healthcare_1     0
Helthcare_2      0
Shops_1          0
Shops_2          0
dtype: int64

In [512]:
# Меняем в test наны на данные с коффициентом по площади и на медианные по здооровью
test.loc[test['Healthcare_1'].isna(), 'Healthcare_1']=aa
test.loc[test['LifeSquare'].isna(), 'LifeSquare'] = test['Square']*ss
test.isna().sum(axis =0)

Id               0
DistrictId       0
Rooms            0
Square           0
LifeSquare       0
KitchenSquare    0
Floor            0
HouseFloor       0
HouseYear        0
Ecology_1        0
Ecology_2        0
Ecology_3        0
Social_1         0
Social_2         0
Social_3         0
Healthcare_1     0
Helthcare_2      0
Shops_1          0
Shops_2          0
dtype: int64

In [513]:
#Анализируем площадь кухни
kitchsqu=test[test['KitchenSquare'] >= 4].sort_values(by = 'KitchenSquare', axis=0)
kitchsqu=kitchsqu[kitchsqu['KitchenSquare'] <= 24].sort_values(by = 'KitchenSquare', axis=0)
kitchsqu
#проверяем корреляцию кухни с другими признаками, площадь, этажность дома, год постройки поэтому из наиболее представительной выборки
#с метражем кухни от 4 до 16 посчитаем коэффициент и применим к остальной выборке
#

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Ecology_2,Ecology_3,Social_1,Social_2,Social_3,Healthcare_1,Helthcare_2,Shops_1,Shops_2
2563,5468,17,1.0,29.080105,21.954910,4.0,3.0,8.0,1960,0.000000,B,B,25,5027,4,46.0,1,1,B
2996,832,33,1.0,32.394741,19.230358,4.0,4.0,5.0,1961,0.051942,B,B,17,4017,7,800.0,2,0,B
303,458,114,1.0,32.466914,17.201054,4.0,8.0,12.0,1966,0.327018,B,B,12,2270,23,2500.0,1,0,B
863,4099,170,2.0,43.298627,28.625901,4.0,9.0,9.0,1968,0.218440,B,B,21,4472,6,1450.0,4,7,A
3909,1397,76,3.0,221.138768,118.055342,4.0,32.0,48.0,2011,0.000000,B,B,7,1660,39,1786.0,1,5,B
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3451,11193,12,1.0,42.901789,19.755764,19.0,7.0,16.0,1980,0.090799,B,B,74,19083,2,990.0,5,15,B
4845,8165,62,3.0,139.101523,90.080906,20.0,3.0,3.0,2015,0.072158,B,B,2,629,1,990.0,0,0,A
4975,5346,53,4.0,110.366487,77.705849,20.0,9.0,24.0,2018,0.049637,B,B,34,7759,0,229.0,1,3,B
2140,4047,168,3.0,92.020293,42.533389,22.0,2.0,2.0,2018,0.341072,B,B,27,5664,48,2300.0,3,11,B


In [514]:
# Расчет среднего коэффициента соотношения площади всего и кухни
zz=(kitchsqu['KitchenSquare'] / kitchsqu['Square']).mean()
zz
#кухня в среднем = 15 процентов от общей площади - будем использовать этот коэффициент

0.15141217893061223

In [515]:
# Меняем в test кухни с площадью меньше 4 и больше 24 на данные с коффициентом zz по общей площади 
#kitchsqu=train[train['KitchenSquare'] >= 4].sort_values(by = 'KitchenSquare', axis=0)
#kitchsqu=kitchsqu[kitchsqu['KitchenSquare'] <= 16].sort_values(by = 'KitchenSquare', axis=0)
#kitchsqu
test.loc[test['KitchenSquare'] <= 4, 'KitchenSquare']=test['Square']*zz
test.loc[test['KitchenSquare'] >= 16, 'KitchenSquare']=test['Square']*zz
test.describe()

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Social_1,Social_2,Social_3,Healthcare_1,Helthcare_2,Shops_1
count,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.00000,5000.000000
mean,8412.595400,51.279200,1.910000,56.449500,36.780726,8.277892,7.838000,12.601000,1984.392600,0.119874,24.933800,5406.900000,8.262600,1072.182400,1.31940,4.242800
std,4832.674037,44.179466,0.838594,19.092787,16.869032,2.689030,5.454989,6.789213,18.573149,0.120070,17.532202,4026.614773,23.863762,760.666726,1.47994,4.777365
min,1.000000,0.000000,0.000000,1.378543,0.333490,0.208728,0.000000,0.000000,1908.000000,0.000000,0.000000,168.000000,0.000000,0.000000,0.00000,0.000000
25%,4221.750000,21.000000,1.000000,41.906231,24.759799,6.000000,4.000000,9.000000,1973.000000,0.019509,6.000000,1564.000000,0.000000,810.000000,0.00000,1.000000
50%,8320.500000,37.000000,2.000000,52.921340,33.737820,8.000000,7.000000,12.000000,1977.000000,0.072158,25.000000,5285.000000,2.000000,990.000000,1.00000,3.000000
75%,12598.250000,77.000000,2.000000,66.285129,45.211111,10.000000,11.000000,17.000000,2000.000000,0.195781,36.000000,7287.000000,5.000000,990.000000,2.00000,6.000000
max,16795.000000,212.000000,17.000000,223.453689,303.071094,33.833610,46.000000,99.000000,2020.000000,0.521867,74.000000,19083.000000,141.000000,4849.000000,6.00000,23.000000


In [516]:
#Выбираем ненулевую площадь для расчета коэфф-та
square2=test[test['Square'] >= test['LifeSquare']].sort_values(by = 'Square', axis=0)
# Расчет среднего коэффициента соотношения площади всего и жилой площади
vv=(square2['LifeSquare'] / square2['Square']).mean()
#заменяем в выборке не верно указанную жилую площадь
test.loc[test['Square'] <= test['LifeSquare'], 'LifeSquare']=test['Square']*vv
#проверяем
square3=test[test['Square'] >= test['LifeSquare']].sort_values(by = 'Square', axis=0) 
square3.head()   

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Ecology_2,Ecology_3,Social_1,Social_2,Social_3,Healthcare_1,Helthcare_2,Shops_1,Shops_2
66,9011,53,1.0,1.378543,1.353573,0.208728,1.0,1.0,1977,0.049637,B,B,34,7759,0,229.0,1,3,B
4902,1420,45,1.0,1.975769,1.241824,0.299155,1.0,1.0,1977,0.195781,B,B,23,5212,6,990.0,3,2,B
4490,1165,27,1.0,2.372101,1.899119,0.359165,2.0,17.0,1977,0.011654,B,B,4,915,0,990.0,0,0,B
608,16401,30,1.0,2.645046,1.662483,0.400492,1.0,1.0,1977,0.000078,B,B,22,6398,141,1046.0,3,23,B
4713,170,6,2.0,2.900586,1.823096,0.439184,17.0,17.0,2014,0.243205,B,B,5,1564,0,540.0,0,0,B


In [517]:
# Убираем буквы в признаках
test.loc[test['Ecology_2'] == 'A', 'Ecology_2'] = 0
test.loc[test['Ecology_2'] == 'B', 'Ecology_2'] = 1
test.loc[test['Ecology_3'] == 'A', 'Ecology_3'] = 0
test.loc[test['Ecology_3'] == 'B', 'Ecology_3'] = 1
test.loc[test['Shops_2'] == 'A', 'Shops_2'] = 0
test.loc[test['Shops_2'] == 'B', 'Shops_2'] = 1
#Смена типа данных в выборке
test['Ecology_2']=test['Ecology_2'].astype(int)
test['Ecology_3']=test['Ecology_3'].astype(int)
test['Shops_2']=test['Shops_2'].astype(int)
test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5000 entries, 0 to 4998
Data columns (total 19 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             5000 non-null   int64  
 1   DistrictId     5000 non-null   int64  
 2   Rooms          5000 non-null   float64
 3   Square         5000 non-null   float64
 4   LifeSquare     5000 non-null   float64
 5   KitchenSquare  5000 non-null   float64
 6   Floor          5000 non-null   float64
 7   HouseFloor     5000 non-null   float64
 8   HouseYear      5000 non-null   int64  
 9   Ecology_1      5000 non-null   float64
 10  Ecology_2      5000 non-null   int32  
 11  Ecology_3      5000 non-null   int32  
 12  Social_1       5000 non-null   int64  
 13  Social_2       5000 non-null   int64  
 14  Social_3       5000 non-null   int64  
 15  Healthcare_1   5000 non-null   float64
 16  Helthcare_2    5000 non-null   int64  
 17  Shops_1        5000 non-null   int64  
 18  Shops_2 

In [518]:
x_tester=pd.DataFrame(test, columns=[ 'DistrictId', 'Rooms', 'Square', 'LifeSquare', 'KitchenSquare',
       'Floor', 'HouseFloor', 'HouseYear', 'Ecology_1', 'Ecology_2',
       'Ecology_3', 'Social_1', 'Social_2', 'Social_3', 'Healthcare_1',
       'Helthcare_2', 'Shops_1', 'Shops_2'])

In [519]:
#Обучаем TEST на полной выборке TRAIN
y_pred3 = clf2.predict(x_tester)
y_pred3


array([157670.93336372, 217565.73763652, 335243.50668824, ...,
       197116.2271588 , 331430.30812379, 213828.73608525])

In [520]:
test['Price']= y_pred3
test.head()

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Ecology_2,Ecology_3,Social_1,Social_2,Social_3,Healthcare_1,Helthcare_2,Shops_1,Shops_2,Price
0,725,58,2.0,49.882643,33.432782,6.000000,6.0,14.0,1972,0.310199,1,1,11,2748,1,990.0,0,0,1,157670.933364
2,5480,190,1.0,13.597819,8.546594,12.000000,2.0,5.0,1909,0.000000,1,1,30,7538,87,4702.0,5,5,1,217565.737637
3,15664,47,2.0,73.046609,51.940842,9.000000,22.0,22.0,2007,0.101872,1,1,23,4583,3,990.0,3,3,1,335243.506688
4,14275,27,1.0,47.527111,43.387569,7.196183,17.0,17.0,2017,0.072158,1,1,2,629,1,990.0,0,0,0,142817.657285
5,7633,53,1.0,40.675627,26.341173,6.158785,21.0,21.0,1977,0.049637,1,1,34,7759,0,229.0,1,3,1,207732.020028


In [521]:
test[['Id', 'Price']].to_csv('ALisov_predictions.csv', index=False)

In [522]:
test[['Id', 'Price']]

,Id,Price
0,725,157670.933364
2,5480,217565.737637
3,15664,335243.506688
4,14275,142817.657285
5,7633,207732.020028
...,...,...
4976,4723,312215.798643
4989,2591,194162.854993
4993,14562,197116.227159
4997,5783,331430.308124
